### Qgis get line intersection

```
processing.run("native:lineintersections", { 'INPUT' : 'G:/809_add_subway_train/linestrings_buildings.shp', 'INPUT_FIELDS' : ['FID','vertex_ind'], 'INTERSECT' : 'G:/809_add_subway_train/bus_line.shp', 'INTERSECT_FIELDS' : ['relation_i','relation_n','osm_id'], 'INTERSECT_FIELDS_PREFIX' : '', 'OUTPUT' : 'G:/807_centerline_xml/bus_intersection.shp' })

processing.run("native:lineintersections", { 'INPUT' : 'G:/809_add_subway_train/linestrings_buildings.shp', 'INPUT_FIELDS' : ['FID','vertex_ind'], 'INTERSECT' : 'G:/809_add_subway_train/subway_line.shp', 'INTERSECT_FIELDS' : ['relation_i','relation_n','osm_id'], 'INTERSECT_FIELDS_PREFIX' : '', 'OUTPUT' : 'G:/807_centerline_xml/subway_intersection.shp' })

processing.run("native:lineintersections",{ 'INPUT' : 'G:/809_add_subway_train/linestrings_buildings.shp', 'INPUT_FIELDS' : ['FID','vertex_ind'], 'INTERSECT' : 'G:/809_add_subway_train/train_line.shp', 'INTERSECT_FIELDS' : ['relation_i','relation_n','osm_id'], 'INTERSECT_FIELDS_PREFIX' : '', 'OUTPUT' : 'G:/807_centerline_xml/train_intersection.shp' })

```

### convert shp to csv to xml

In [1]:
import csv

In [2]:
dict_bus ={}
with open ("bus_intersection.csv",'rt',encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader, None)
    for row in reader :
        WKT,FID,vertex_ind,relation_i,relation_n,osm_id = row
        if FID not in dict_bus.keys():
            dict_osm = {}
            dict_osm [osm_id] = [{relation_i:relation_n}]
            dict_bus[FID]= dict_osm
        else:
            if osm_id in dict_bus[FID].keys():
                if {relation_i:relation_n} not in dict_bus[FID][osm_id] :
                    dict_bus[FID][osm_id].append({relation_i:relation_n})
            else:
                dict_osm = {}
                dict_osm [osm_id] = [{relation_i:relation_n}]
                dict_bus[FID]= dict_osm
                

In [3]:
for key,value in dict_bus.items():
    print(key,value)
    break

14 {'283473009': [{'1253894': 'Bus 72 : Gare de Lyon → Parc de Saint-Cloud'}]}


In [4]:
dict_subway ={}
with open ("subway_intersection.csv",'rt',encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader, None)
    for row in reader :
        WKT,FID,vertex_ind,relation_i,relation_n,osm_id = row
        if FID not in dict_subway.keys():
            dict_osm = {}
            dict_osm [osm_id] = [{relation_i:relation_n}]
            dict_subway[FID]= dict_osm
        else:
            if osm_id in dict_subway[FID].keys():
                if {relation_i:relation_n} not in dict_subway[FID][osm_id] :
                    dict_subway[FID][osm_id].append({relation_i:relation_n})
            else:
                dict_osm = {}
                dict_osm [osm_id] = [{relation_i:relation_n}]
                dict_subway[FID]= dict_osm                          

In [5]:
for key,value in dict_subway.items():
    print(key,value)
    break

15 {'24380725': [{'123768': "Métro 10: Boulogne - Pont de Saint-Cloud → Gare d'Austerlitz"}, {'3328740': "Métro 10: Gare d'Austerlitz → Boulogne - Pont de Saint-Cloud"}]}


In [6]:
dict_train ={}
with open ("train_intersection.csv",'rt',encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader, None)
    for row in reader :
        WKT,FID,vertex_ind,relation_i,relation_n,osm_id = row
        if FID not in dict_train.keys():
            dict_osm = {}
            dict_osm [osm_id] = [{relation_i:relation_n}]
            dict_train[FID]= dict_osm
        else:
            if osm_id in dict_train[FID].keys():
                if {relation_i:relation_n} not in dict_train[FID][osm_id] :
                    dict_train[FID][osm_id].append({relation_i:relation_n})
            else:
                dict_osm = {}
                dict_osm [osm_id] = [{relation_i:relation_n}]
                dict_train[FID]= dict_osm
                
                                                  
           

In [7]:
for key,value in dict_train.items():
    print(key,value)
    break

711 {'181567868': [{'3047780': 'RER C Tronçon C7 - Champ de Mars - Tour Eiffel → Saint-Quentin-en-Yvelines'}, {'3049185': 'RER C Tronçon C5 - Champ de Mars - Tour Eiffel → Versailles Rive Gauche'}]}


In [8]:
import xml.etree.ElementTree as ET

In [9]:
centerline_tree = ET.parse('centerline.xml')
centerline_root = centerline_tree.getroot()
centerline_nodes= centerline_root.find('nodes')
centerline_links= centerline_root.find('links')

In [10]:
for centerline_link in centerline_links.findall('link'):
    linkid = centerline_link.get('linkId')
    
    if linkid in dict_bus.keys():
        for way_id , ligne in dict_bus[linkid].items():
            for lg in ligne:
                for relation_i, relation_n in lg.items():
                    parametre ='bus'
                    ligne_name=relation_n.split(' ')
                    osm_parametre = centerline_link.find(parametre)
                    if (osm_parametre!=None):
                        osm_parametre.set(parametre, osm_parametre.get(parametre)+' '+ 'lingeId:'+relation_i+','+ligne_name[0]+':'+ligne_name[1])
                    else:
                        ET.SubElement(centerline_link, parametre,{parametre: 'lingeId:'+relation_i+','+ligne_name[0]+':'+ligne_name[1]})
    
    if linkid in dict_subway.keys():
        for way_id , ligne in dict_subway[linkid].items():
            for lg in ligne:
                for relation_i, relation_n in lg.items():
                    parametre ='subway'
                    ligne_name=relation_n.split(' ')
                    osm_parametre = centerline_link.find(parametre)
                    if (osm_parametre!=None):
                        osm_parametre.set(parametre, osm_parametre.get(parametre)+' '+ 'lingeId:'+relation_i+','+ligne_name[0]+':'+ligne_name[1])
                    else:
                        ET.SubElement(centerline_link, parametre,{parametre: 'lingeId:'+relation_i+','+ligne_name[0]+':'+ligne_name[1]})
                        
    if linkid in dict_train.keys():
        for way_id , ligne in dict_train[linkid].items():
            for lg in ligne:
                for relation_i, relation_n in lg.items():
                    parametre ='train'
                    ligne_name=relation_n.split(' ')
                    osm_parametre = centerline_link.find(parametre)
                    if (osm_parametre!=None):
                        osm_parametre.set(parametre, osm_parametre.get(parametre)+' '+ 'lingeId:'+relation_i+','+ligne_name[0]+':'+ligne_name[1])
                    else:
                        ET.SubElement(centerline_link, parametre,{parametre: 'lingeId:'+relation_i+','+ligne_name[0]+':'+ligne_name[1]})

In [11]:
tree = ET.ElementTree(centerline_root)
ET.indent(tree)
tree.write('output.xml',encoding='utf-8')

```
    <link linkId="1391" length="122.574" node1="58222" node2="58335">
      <nodes nodeId="58222 58223 58224 ..." />
      <osm osm_id="24642237 749622104 24030143 24642446" />
      <width width="24.859735642220222 20.050516735498142 20.30311626228249 27.78647952544148 37.16374291122703" />
      <freespeed freespeed="8.333333333333334 8.333333333333334 8.333333333333334 8.333333333333334" />
      <capacity capacity="480.0 1440.0 480.0 800.0" />
      <permlanes permlanes="1.0 3.0 1.0 1.0" />
      <oneway oneway="1 1 1 1" />
      <modes modes="car,pt,bus,car_passenger car,pt,bus,car_passenger car,car_passenger car,pt,bus,car_passenger" />
      <bus bus="lingeId:1255722,Bus:32 lingeId:1255726,Bus:32 lingeId:1709734,Noctilien:N53 lingeId:6434463,Noctilien:N53" />
      <subway subway="lingeId:123782,Métro:9 lingeId:3328716,Métro:9" />
      <train train="lingeId:3014389,RER:C lingeId:3027737,RER:C" />
    </link>
```